In [204]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

### Kombinasi 9 :
* Delete Duplicate
* Delete Null
* Outlier capping winsorize sisi kanan
* Encoding 
* Standard scaler
* Feature selection -> K-Best

In [205]:
df = pd.read_csv('../../Without Feature Selection/UFC_kombinasi9_all_features.csv')

In [206]:
X = df.drop(['B_Reach_cms'], axis=1)
y = df['B_Reach_cms']

In [207]:
df_test = pd.read_csv('../../regression_kaggle/UFC_kombinasi9_all_features.csv')
df_test = df_test.drop(['B_Reach_cms'], axis=1, errors='ignore')
df_test_id = df_test['id']
df_test = df_test.drop(['id'], axis=1, errors='ignore')
# Get the common columns between df and df_test
common_columns = list(set(X.columns) & set(df_test.columns))
# Update df_test to only include the common columns
df_test = df_test[common_columns]
X = X[common_columns]

In [208]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [209]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras import backend as K
from keras.regularizers import l1

# Define R-squared metric
def r_squared(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return (1 - SS_res/(SS_tot + K.epsilon()))

# Define the model architecture
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(X.shape[1],), kernel_regularizer=l1(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu', kernel_regularizer=l1(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu', kernel_regularizer=l1(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(1))

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[r_squared])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=20)

# Define learning rate scheduler
def scheduler(epoch, lr):
  if epoch < 10:
    return lr
  else:
    return lr * tf.math.exp(-0.1)

lr_scheduler = LearningRateScheduler(scheduler)

# Train the model
model.fit(X, y, epochs=300, batch_size=32, validation_split=0.2, callbacks=[early_stopping, lr_scheduler])

# Evaluate the model
loss, r2_score = model.evaluate(X, y)
print("R-squared:", r2_score)

Epoch 1/300
87/87 [==============================] - 1s 7ms/step - loss: 33341.0547 - r_squared: -416.0704 - val_loss: 32985.9492 - val_r_squared: -407.2169 - lr: 0.0010
Epoch 2/300
87/87 [==============================] - 0s 6ms/step - loss: 32398.9883 - r_squared: -406.0000 - val_loss: 31548.4688 - val_r_squared: -389.2720 - lr: 0.0010
Epoch 3/300
87/87 [==============================] - 1s 6ms/step - loss: 30304.5195 - r_squared: -384.3315 - val_loss: 28830.6816 - val_r_squared: -355.6681 - lr: 0.0010
Epoch 4/300
87/87 [==============================] - 1s 6ms/step - loss: 26738.9082 - r_squared: -338.1450 - val_loss: 24605.0176 - val_r_squared: -303.1652 - lr: 0.0010
Epoch 5/300
87/87 [==============================] - 1s 6ms/step - loss: 22109.9590 - r_squared: -278.3349 - val_loss: 19298.2793 - val_r_squared: -237.2485 - lr: 0.0010
Epoch 6/300
87/87 [==============================] - 1s 6ms/step - loss: 16885.4980 - r_squared: -206.5498 - val_loss: 14333.0938 - val_r_squared: -17

In [210]:
# import tensorflow as tf
# from keras.models import Sequential
# from keras.layers import Dense, Dropout, BatchNormalization
# from keras.callbacks import EarlyStopping
# from keras import backend as K
# from keras.regularizers import l1

# # Define R-squared metric
# def r_squared(y_true, y_pred):
#     SS_res =  K.sum(K.square(y_true - y_pred)) 
#     SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
#     return (1 - SS_res/(SS_tot + K.epsilon()))

# # Define the model architecture
# model = Sequential()
# model.add(Dense(512, activation='relu', input_shape=(X.shape[1],), activity_regularizer=l1(0.001)))
# model.add(BatchNormalization())
# model.add(Dropout(0.3))
# model.add(Dense(256, activation='relu', activity_regularizer=l1(0.001)))
# model.add(BatchNormalization())
# model.add(Dropout(0.3))
# model.add(Dense(128, activation='relu', activity_regularizer=l1(0.001)))
# model.add(BatchNormalization())
# model.add(Dropout(0.3))
# model.add(Dense(1))

# # Compile the model
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
# model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[r_squared])

# # Define early stopping
# early_stopping = EarlyStopping(monitor='val_loss', patience=15)

# # Train the model
# model.fit(X, y, epochs=300, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# # Evaluate the model
# loss, r2_score = model.evaluate(X, y)
# print("R-squared:", r2_score)

In [211]:
df_test_scaled = scaler.transform(df_test)
y_pred = model.predict(df_test_scaled)

25/25 [==============================] - 0s 1ms/step


In [212]:
predictions_df = pd.DataFrame(y_pred, columns=['Predicted'])

In [213]:
predictions_df['id'] = df_test_id
submission = predictions_df[['id', 'Predicted']]

In [214]:
submission.head()

,id,Predicted
0,0.0,158.396530
1,1.0,161.924149
2,2.0,189.968018
3,3.0,165.441910
4,4.0,169.253510


In [215]:
submission.to_csv('pred_kombinasi3_neural_network_test2.csv', index=False)